# Trainingspipeline 20

- balanciertes Korpus
- ohne Bigramme
- vector_size: 300
- window: 5
- seed: 1

In [1]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

from gensim.models import Word2Vec
from joblib import Parallel, delayed  
from nltk.corpus import stopwords

In [2]:
stopwords = stopwords.words('italian')
tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [3]:
df = pd.read_csv('../Korpus/Korpus/corpus_balanced.csv', sep=',', encoding='utf-8')

In [4]:
df.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text,cleaned_tokenized_text
0,Poesia.IV.4.Testo.txt,MIDIA,Faustina Maratti Zappi,Poesie,1700.0,1700-1750,poesia,IV. 4. Rime degli Arcadi: Aglauro Cidonia (Fau...,3184.0,iv . 4 . rima del arcadi : aglauro cidonia ( f...,"[['iv'], [], ['rima', 'arcadi', 'aglauro', 'ci..."
1,Espositivi.IV.4.Testo.txt,MIDIA,Ludovico Antonio Muratori,Antichità italiane,1700.0,1700-1750,espositivo,"﻿IV. 4. Ludovico Antonio Muratori, Antichità i...",8990.0,"﻿iv . 4 . Ludovico Antonio muratori , antichit...","[['iv'], [], ['ludovico', 'antonio', 'muratori..."
2,Personali.IV.5.Testo.txt,MIDIA,Lorenzo Magalotti,Lettere odorose (1693-1705),1700.0,1700-1750,personale,"IV. 5. Lorenzo Magalotti, Lettere odorose (169...",8374.0,"iv . 5 . Lorenzo magalotti , lettere odoroso (...","[['iv'], [], ['lorenzo', 'magalotti', 'lettere..."
3,Personali.IV.15.Testo.txt,MIDIA,Pietro Giannone,Vita scritta da lui medesimo,1700.0,1700-1750,personale,[Proemio]\nPrendo a scrivere la mia vita e qua...,10118.0,[ proemio ] \n prendere a scrivere il mio vita...,"[['proemio', 'prendere', 'scrivere', 'vita', '..."
4,Personali.IV.4.Testo.txt,MIDIA,Vincenzo da Filicaia,Lettere inedite a Lorenzo Magalotti,1700.0,1700-1750,personale,"IV. 4. Vincenzo da Filicaia, Lettere inedite a...",10073.0,"iv . 4 . Vincenzo da filicaia , lettere inedit...","[['iv'], [], ['vincenzo', 'filicaia', 'lettere..."


In [5]:
df.shape

(743763, 11)

In [6]:
df.text = df.text.fillna('')
df.lemmatized_text = df.lemmatized_text.fillna('')

In [7]:
# Einzeldataframes für die Zeiträume

df_periods = dict(tuple(df.groupby(by='period')))

In [8]:
df1 = df_periods['1700-1750']
df2 = df_periods['1751-1800']
df3 = df_periods['1801-1825']
df4 = df_periods['1826-1850']
df5 = df_periods['1851-1875']
df6 = df_periods['1876-1900']
df7 = df_periods['1901-1925']
df8 = df_periods['1926-1950']
df9 = df_periods['1951-1975']
df10 = df_periods['1976-2000']
df11 = df_periods['2001-2010']
df12 = df_periods['2011-2016']
df13 = df_periods['2017-2021']

In [9]:
df9.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text,cleaned_tokenized_text
126313,LISRodari3.txt,LIS,Gianno Rodari,La questione dei fumetti,1951.0,1951-1975,stampa,"Caro Direttore , ho letto nell ' ultimo numero...",1510.0,"caro direttore , avere leggere nell ' ultimo n...","[['caro', 'direttore', 'avere', 'leggere', 'ul..."
126314,LISJotti1.txt,LIS,Nilde Jotti,La questione dei fumetti,1951.0,1951-1975,stampa,Il dibattito sulla stampa a fumetti per i raga...,2785.0,il dibattito sulla stampa a fumetto per il rag...,"[['dibattito', 'stampa', 'fumetto', 'ragazzo',..."
126315,LLAlbertelli1.txt,Liber Liber,Pilo Albertelli,Rousseau,1951.0,1951-1975,prosa letteraria,﻿Pilo Albertelli\nRousseau\n\n Nacque il 28 g...,4894.0,﻿pilo albertelli \n rousseau \n\n nascere il...,"[['pilo', 'albertelli', 'rousseau', 'nascere',..."
126316,LISManacorda1.txt,LIS,Gastone Manacorda,Il Partito e la sua funzione di guida nel camp...,1951.0,1951-1975,stampa,"Al partito , nel suo rigoglioso sviluppo , seg...",3460.0,"al partito , nel suo rigoglioso sviluppo , seg...","[['partito', 'rigoglioso', 'sviluppo', 'seguit..."
126317,LISBianchi1.txt,LIS,Ranuccio Bianchi Bandinelli,Il nostro lavoro nella scuola,1951.0,1951-1975,stampa,"Come in tutti i congressi , anche nel VII Cong...",2898.0,"come in tutto il congresso , anche nel vii con...","[['congresso', 'vii', 'congresso', 'p'], ['tes..."


## Training von Word2Vec

In [10]:
# Hilfsfunktionen zur Vorbereitung auf das Training
# Bereinigung und Tokenisierung

def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_àÀèÈìÌòÒùÙáÁéÉíÍóÓúÚ]',' ', raw).split()        # Diakritika ans Italienische anpassen                    
    filtered_text = [word for word in text if word not in stopwords]        # Stopwörter löschen
    return filtered_text


def tokenize_text(raw_text):
    """
    returns a list of lowercase tokenized sentences 
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    sentences = tokenized_sentences
    return sentences

In [11]:
# Trainingsparamter setzen

vector_size = 300                  # Dimensionality of the word vectors
window = 5                         # The maximum distance between the current and predicted word within a sentence
min_count = 2                      # (int, optional) – The model ignores all words with total frequency lower than this
workers = 1                        # Use these many worker threads to train the model (faster training with multicore machines)
min_alpha = 0.0001                 # Learning rate will linearly drop to min_alpha as training progresses
sg = 1                             # Training algorithm: skip-gram if sg=1, otherwise CBOW            
seed = 1                           # Reproductivity --> only if workers = 1

In [12]:
# Ordner anlegen zum Abspeichern von trainierten Modellen

if not os.path.exists('../trained_models'):
    os.makedirs('../trained_models')

### Zeitraum 1: 1700-1750

In [13]:
# lemmatisierte Texte zu einem String verbinden

text1 = ''

for i in df1.lemmatized_text:
    text1 += i

In [14]:
%%time
sentences1 = tokenize_text(text1)         # Bereinigen, Tokenisieren und in Form bringen (Ziel: Liste von tokenisierten Sätzen)

Wall time: 22 s


In [15]:
%%time

# Training   

w2v1 = Word2Vec(sentences=sentences1,                      
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 34s


In [16]:
w2v1.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.6440621614456177),
 ('goti', 0.5823515057563782),
 ('costernazione', 0.5814597010612488),
 ('addossare', 0.5808659195899963),
 ('germani', 0.573516309261322),
 ('riempiè', 0.5556989908218384),
 ('alamanni', 0.5554428696632385),
 ('turchesche', 0.5527563095092773),
 ('imminente', 0.5485216975212097),
 ('barbari', 0.546453595161438)]

In [17]:
# trainiertes Modell speichern

w2v1.save(os.path.join('../trained_models/Word2Vec20', '20w2v1.model'))

### Zeitraum 2: 1751-1800

In [20]:
text2 = ''

for i in df2.lemmatized_text:
    text2 += i

In [21]:
%%time
sentences2 = tokenize_text(text2)

Wall time: 25.2 s


In [22]:
%%time
w2v2 = Word2Vec(sentences=sentences2,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 33s


In [23]:
w2v2.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.6144841313362122),
 ('invisibili', 0.6141772270202637),
 ('ostile', 0.6009544134140015),
 ('formidabile', 0.5985737442970276),
 ('inspirare', 0.5967345237731934),
 ('formidabil', 0.5905941128730774),
 ('devastazione', 0.5898666381835938),
 ('crociati', 0.5879931449890137),
 ('legione', 0.5832064747810364),
 ('goto', 0.5808111429214478)]

In [24]:
w2v2.save(os.path.join('../trained_models/Word2Vec20', '20w2v2.model'))

### Zeitraum 3: 1801-1825

In [25]:
text3 = ''

for i in df3.lemmatized_text:
    text3 += i

In [26]:
%%time
sentences3 = tokenize_text(text3)

Wall time: 22.4 s


In [27]:
%%time
w2v3 = Word2Vec(sentences=sentences3,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 31s


In [28]:
w2v3.wv.most_similar(positive=['terrore'], topn=10)

[('repentino', 0.7212904691696167),
 ('sorpresa', 0.6679139733314514),
 ('aspettò', 0.6589752435684204),
 ('perturbazione', 0.656281590461731),
 ('imaginari', 0.6494259238243103),
 ('disperazione', 0.6442292332649231),
 ('inesprimibile', 0.6415088772773743),
 ('orrore', 0.6396117806434631),
 ('tremito', 0.6388493180274963),
 ('spavento', 0.6373303532600403)]

In [29]:
w2v3.save(os.path.join('../trained_models/Word2Vec20', '20w2v3.model'))

### Zeitraum 4: 1826-1850

In [30]:
text4 = ''

for i in df4.lemmatized_text:
    text4 += i

In [31]:
%%time
sentences4 = tokenize_text(text4)

Wall time: 21.9 s


In [32]:
%%time
w2v4 = Word2Vec(sentences=sentences4,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 24s


In [33]:
w2v4.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.5500614643096924),
 ('bentosto', 0.5425611138343811),
 ('panico', 0.5242947340011597),
 ('inspirare', 0.520978569984436),
 ('cagionare', 0.4957844316959381),
 ('incusso', 0.48901456594467163),
 ('buonaparte', 0.4864822030067444),
 ('maraviglia', 0.4791521430015564),
 ('stupore', 0.47347643971443176),
 ('riurto', 0.4715731739997864)]

In [34]:
w2v4.save(os.path.join('../trained_models/Word2Vec20', '20w2v4.model'))

### Zeitraum 5: 1851-1875

In [35]:
text5 = ''

for i in df5.lemmatized_text:
    text5 += i

In [36]:
%%time
sentences5 = tokenize_text(text5)

Wall time: 23.9 s


In [37]:
%%time
w2v5 = Word2Vec(sentences=sentences5,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 23s


In [38]:
w2v5.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.616623044013977),
 ('orrore', 0.605890154838562),
 ('incutere', 0.6041738390922546),
 ('sdegno', 0.5938243865966797),
 ('raccapriccio', 0.5908360481262207),
 ('sgomento', 0.5889360308647156),
 ('rabbia', 0.582414984703064),
 ('ribrezzo', 0.580986499786377),
 ('scoraggiamento', 0.5757744312286377),
 ('orrendo', 0.5679986476898193)]

In [39]:
w2v5.save(os.path.join('../trained_models/Word2Vec20', '20w2v5.model'))

### Zeitraum 6: 1876-1900

In [40]:
text6 = ''

for i in df6.lemmatized_text:
    text6 += i

In [41]:
%%time
sentences6 = tokenize_text(text6)

Wall time: 26.7 s


In [42]:
%%time
w2v6 = Word2Vec(sentences=sentences6,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 14s


In [43]:
w2v6.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.5963409543037415),
 ('sgomento', 0.5857391357421875),
 ('orrore', 0.4911809265613556),
 ('raccapriccio', 0.4715091288089752),
 ('timore', 0.4638584852218628),
 ('stupore', 0.46026942133903503),
 ('indicibile', 0.4569336473941803),
 ('sdegno', 0.45664846897125244),
 ('furore', 0.4530036747455597),
 ('ribrezzo', 0.4522262215614319)]

In [44]:
w2v6.save(os.path.join('../trained_models/Word2Vec20', '20w2v6.model'))

### Zeitraum 7: 1901-1925

In [45]:
text7 = ''

for i in df7.lemmatized_text:
    text7 += i

In [46]:
%%time
sentences7 = tokenize_text(text7)

Wall time: 26.8 s


In [47]:
%%time
w2v7 = Word2Vec(sentences=sentences7,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 17s


In [48]:
w2v7.wv.most_similar(positive=['terrore'], topn=10)

[('sgomento', 0.5704684257507324),
 ('smarrimento', 0.5281331539154053),
 ('orrore', 0.5271201133728027),
 ('raccapriccio', 0.5173725485801697),
 ('ribrezzo', 0.5141361355781555),
 ('spavento', 0.5056495666503906),
 ('giovanezza', 0.49964988231658936),
 ('esaltazione', 0.4897889494895935),
 ('disgusto', 0.4892027974128723),
 ('accesso', 0.488399863243103)]

In [49]:
w2v7.save(os.path.join('../trained_models/Word2Vec20', '20w2v7.model'))

### Zeitraum 8: 1926-1950

In [50]:
text8 = ''

for i in df8.lemmatized_text:
    text8 += i

In [51]:
%%time
sentences8 = tokenize_text(text8)

Wall time: 26.1 s


In [52]:
%%time
w2v8 = Word2Vec(sentences=sentences8,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 24s


In [53]:
w2v8.wv.most_similar(positive=['terrore'], topn=10)

[('rabbico', 0.624796450138092),
 ('virus', 0.6029857993125916),
 ('paralizzare', 0.529996395111084),
 ('repentino', 0.527737557888031),
 ('subitaneo', 0.5202751755714417),
 ('raccapriccio', 0.5193615555763245),
 ('impietrire', 0.5113577246665955),
 ('stupore', 0.5038285851478577),
 ('pettirosso', 0.49877503514289856),
 ('pazza', 0.4941707253456116)]

In [54]:
w2v8.save(os.path.join('../trained_models/Word2Vec20', '20w2v8.model'))

### Zeitraum 9: 1951-1975

In [55]:
text9 = ''

for i in df9.lemmatized_text:
    text9 += i

In [56]:
%%time
sentences9 = tokenize_text(text9)

Wall time: 23.3 s


In [57]:
%%time
w2v9 = Word2Vec(sentences=sentences9,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 21s


In [58]:
w2v9.wv.most_similar(positive=['terrore'], topn=10)

[('smarrimento', 0.6900624632835388),
 ('tremendo', 0.6719077825546265),
 ('orrore', 0.6677804589271545),
 ('immane', 0.6637953519821167),
 ('orrendo', 0.663218080997467),
 ('presagio', 0.6616920828819275),
 ('terrificante', 0.6592441201210022),
 ('oppressione', 0.6584136486053467),
 ('delirio', 0.6571167707443237),
 ('sconforto', 0.6542895436286926)]

In [59]:
w2v9.save(os.path.join('../trained_models/Word2Vec20', '20w2v9.model'))

### Zeitraum 10: 1976-2000

In [60]:
text10 = ''

for i in df10.lemmatized_text:
    text10+= i

In [61]:
%%time
sentences10 = tokenize_text(text10)

Wall time: 27 s


In [62]:
%%time
w2v10 = Word2Vec(sentences=sentences10,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 19s


In [63]:
w2v10.wv.most_similar(positive=['terrore'], topn=10)

[('terrorismo', 0.5661575198173523),
 ('incubo', 0.5597116947174072),
 ('strage', 0.5499250292778015),
 ('stalin', 0.547441303730011),
 ('persecuzione', 0.5441486835479736),
 ('mietere', 0.5359048247337341),
 ('follia', 0.5319666862487793),
 ('criminale', 0.5312025547027588),
 ('pol', 0.5309472680091858),
 ('peste', 0.5280770659446716)]

In [64]:
w2v10.save(os.path.join('../trained_models/Word2Vec20', '20w2v10.model'))

### Zeitraum 11: 2001-2010

In [65]:
text11 = ''

for i in df11.lemmatized_text:
    text11+= i

In [66]:
%%time
sentences11 = tokenize_text(text11)

Wall time: 21.3 s


In [67]:
%%time
w2v11 = Word2Vec(sentences=sentences11,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 37s


In [68]:
w2v11.wv.most_similar(positive=['terrore'], topn=10)

[('orrore', 0.6102033257484436),
 ('nemico', 0.6040517687797546),
 ('terrorismo', 0.5963135957717896),
 ('odio', 0.5962340831756592),
 ('barbarie', 0.5942692160606384),
 ('ceceno', 0.5938518047332764),
 ('oppressione', 0.586752712726593),
 ('feroce', 0.5840008854866028),
 ('smarrimento', 0.5810507535934448),
 ('mostro', 0.5789194107055664)]

In [69]:
w2v11.save(os.path.join('../trained_models/Word2Vec20', '20w2v11.model'))

### Zeitraum 12: 2011-2016

In [70]:
text12 = ''

for i in df12.lemmatized_text:
    text12+= i

In [71]:
%%time
sentences12 = tokenize_text(text12)

Wall time: 19.7 s


In [72]:
%%time
w2v12 = Word2Vec(sentences=sentences12,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 41s


In [73]:
w2v12.wv.most_similar(positive=['terrore'], topn=10)

[('angoscia', 0.6695412993431091),
 ('incutere', 0.657163679599762),
 ('desolazione', 0.6458594799041748),
 ('seminare', 0.6414576172828674),
 ('poliziesco', 0.640405535697937),
 ('paura', 0.6369116902351379),
 ('ribellione', 0.6351642608642578),
 ('attimi', 0.6351029276847839),
 ('miseria', 0.6331980228424072),
 ('ripiombare', 0.6331717371940613)]

In [74]:
w2v12.save(os.path.join('../trained_models/Word2Vec20', '20w2v12.model'))

### Zeitraum 13: 2017-2021

In [75]:
text13 = ''

for i in df13.lemmatized_text:
    text13+= i

In [76]:
%%time
sentences13 = tokenize_text(text13)

Wall time: 21.4 s


In [77]:
%%time
w2v13 = Word2Vec(sentences=sentences13,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 47s


In [78]:
w2v13.wv.most_similar(positive=['terrore'], topn=10)

[('orrore', 0.7440875768661499),
 ('impotenza', 0.7062187194824219),
 ('disperazione', 0.6998205780982971),
 ('seminare', 0.697414755821228),
 ('incutere', 0.6954268217086792),
 ('odio', 0.6937553286552429),
 ('oppressione', 0.6924097537994385),
 ('crudeltà', 0.6852359771728516),
 ('assalire', 0.6844949126243591),
 ('frustrazione', 0.6841525435447693)]

In [79]:
w2v13.save(os.path.join('../trained_models/Word2Vec20', '20w2v13.model'))